# Lab_5

## Zadania #1
**1.1 Napisać odpowiedniki echo3 i dialog wykorzystujące notację do**

In [1]:
echo3 :: IO ()
echo3 = do
    line1 <- getLine
    line2 <- getLine
    putStrLn $ line1 ++ line2

In [2]:
dialog :: IO ()
dialog = do
    putStr "What is your happy number? "
    n <- getLine
    let num = read n :: Int in
                   if num == 7
                   then putStrLn "Ah, lucky 7!"
                   else if odd num
                        then putStrLn "Odd number! That's most people's choice..."
                        else putStrLn "Hm, even number? Unusual!"

**1.2 Napisać odpowiednik twoQuestions bez użycia notacji do**

In [3]:
twoQuestions :: IO ()
twoQuestions = putStr "What is your name? "
            >> getLine
            >>= \n -> putStr "How old are you? "
            >> getLine
            >>= \o -> print (n, o)

**1.3 Napisać ‘akcję’ getLine' odpowiadającą getLine z biblioteki Prelude**

In [4]:
getLine' :: IO String
getLine' = do
    c <- getChar
    if c == '\n' then return ""
    else do
        s <- getLine
        return (c:s)

## Zadania #2
**2.1 Napisać odpowiednik sequence' wykorzystujący foldr**

In [5]:
sequence' :: [IO ()] -> IO ()
sequence' = foldr (>>) (return ())

Line 2: Use sequence_
Found:
foldr (>>) (return ())
Why not:
sequence_

In [6]:
actionList :: [IO ()]
actionList = [putStrLn "Ben",
              putStrLn "Joe",
              putStrLn "Ally"]
              
sequence' actionList

Ben
Joe
Ally

**2.2 Zmienić postać 1. agrumentu foldr: z >> na wyrażenie lambda**

In [7]:
sequence'' :: [IO ()] -> IO ()
sequence'' = foldr (\x xs -> x >> xs) (return ())

sequence'' actionList

Line 2: Avoid lambda
Found:
\ x xs -> x >> xs
Why not:
(>>)

Ben
Joe
Ally

**2.3 Napisać odpowiednik sequence' wykonujący ‘akcje’ od ostaniej do pierwszej; rozważyć co najmniej dwa warianty, np. foldr na odwróconej liście i wykorzystanie foldl**

In [8]:
sequenceRev :: [IO ()] -> IO ()
sequenceRev xs = foldr (>>) (return ()) (reverse xs)

sequenceRev actionList

Line 2: Use sequence_
Found:
foldr (>>) (return ())
Why not:
sequence_

Ally
Joe
Ben

In [9]:
sequenceRev' :: [IO ()] -> IO ()
sequenceRev' = foldl (>>) (return ())

sequenceRev actionList

Ally
Joe
Ben

## Zadania #3
**3.1 Sprawdzić możliwość automatycznego wygenerowania instancji Functor dla typu MyList (klauzula deriving)**

In [10]:
{-# LANGUAGE DeriveFunctor #-}

data MyList a = EmptyList
              | Cons a (MyList a) deriving (Show, Functor)

In [11]:
let lst1 = Cons 1 (Cons 2 (Cons 3 (Cons 4 EmptyList)))
fmap (*2) lst1

Cons 2 (Cons 4 (Cons 6 (Cons 8 EmptyList)))

**3.2 Napisać własną implementację funktora (instance Functor), a następnie sprawdzić możliwość jej automatycznego wygenerowania dla drzewa binarnego zdefiniowanego jako**

In [12]:
data BinTree a = EmptyBT | NodeBT a (BinTree a) (BinTree a) 
            deriving (Show)
            
instance Functor BinTree where
    fmap g EmptyBT = EmptyBT
    fmap g (NodeBT a lt rt) = NodeBT (g a) (fmap g lt) (fmap g rt)

In [13]:
data BinTree' a = EmptyBT' | NodeBT' a (BinTree' a) (BinTree' a) 
            deriving (Show, Functor)

**3.3 Napisać implementacje funktora (instance Functor) dla następujących typów:**

In [14]:
newtype Pair b a = Pair { getPair :: (a,b) } deriving Show
-- fmap should change the first element

instance Functor (Pair b) where
    fmap g Pair { getPair = (a,b) } = Pair {getPair = (g a,b)}
    
fmap (+2) (Pair (1,2))

Pair {getPair = (3,2)}

In [15]:
data Tree2 a = EmptyT2 | Leaf a | Node (Tree2 a) a (Tree2 a) deriving Show

instance Functor Tree2 where
    fmap g EmptyT2 = EmptyT2
    fmap g (Leaf a) = Leaf (g a)
    fmap g (Node lt a rt) = Node (fmap g lt) (g a) (fmap g rt)

In [16]:
data GTree a = Leaf a | GNode [GTree a] deriving Show

instance Functor GTree where
    fmap g (Leaf a) = Leaf (g a)
    fmap g (GNode xs) = GNode (map (fmap g) xs)

# Zadania #4
**4.1 Napisać implementacje funktora aplikatywnego (instance Applicative) dla typu**

In [17]:
newtype MyTriple a = MyTriple (a,a,a) deriving Show

instance Functor MyTriple where
    fmap g (MyTriple (a,b,c)) = MyTriple (g a, g b, g c)
    
instance Applicative MyTriple where
    pure a = MyTriple (a,a,a)
    MyTriple (f,g,h) <*> MyTriple (a,b,c) = MyTriple (f a, g b, h c)